<a href="https://colab.research.google.com/github/itsmuditt/Stock_Price_Prediction/blob/main/News_for_stock_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **WEB SCRAPING**

---
We will scrape headlines from websites using the piece of code in this segment.


---




In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [ ]:
import datetime
start_date = datetime.datetime(2013, 1, 1)

date_list = [start_date.date() + datetime.timedelta(days=x) for x in range(44976-41275)]
dates = [x.strftime("%d-%m-%Y") for x in date_list]


### **Economic Times Website's Archieves**

---


```
# Doing it for Economic Times website's ARCHIEVES
```



In [ ]:
news = []
for starttime in range(41275, 44976):
  url=f'https://economictimes.indiatimes.com/archivelist/starttime-{starttime}.cms'
  response = requests.get(url)

  soup = BeautifulSoup(response.text, 'html.parser')
  headlines = soup.body.find_all('td', class_="contentbox5")
  day = []
  for x in headlines:
    y = x.find_all('a')
    for heads in y:
      temp = heads.text.strip()
      if len(temp.split())>4:
        day.append(temp)

  news.append(day)

In [ ]:
news_et_df = pd.DataFrame(list(zip(dates, news )))

In [ ]:
news_et_df.to_csv('news_et.csv')


### **Times of India Website's Archieves**

---


```
# Doing it for Times of India website's ARCHIEVES
```



In [ ]:
news_toi = []
for starttime in range(41275, 44976):
  url=f'https://timesofindia.indiatimes.com/archivelist/starttime-{starttime}.cms'
  response = requests.get(url)

  soup = BeautifulSoup(response.text, 'html.parser')
  headlines = soup.body.find_all('span')
  day = []
  for x in headlines:
    y = x.find_all('a')
    for heads in y:
      temp = heads.text.strip()
      if len(temp.split())>4:
        day.append(temp)

  news_toi.append(day)

In [ ]:
news_toi_df = pd.DataFrame(list(zip(dates, news_toi)))

In [ ]:
news_toi_df.to_csv('news_toi.csv')

## **Revising our DATASET**

---

```
The dataset we were left with, after web scraping, had all the headlines into one column.

The below segment of code, changes it to having as many columns as the particular row requires.

```



---



In [ ]:
import pandas as pd

In [ ]:
net = pd.read_csv('news_et.csv')

In [ ]:
ntoi = pd.read_csv('news_toi.csv')

In [ ]:
def split_sentences(row):
    max_sentences = len(row['Headlines'])
    cnt = 1
    i = 2
    while i < max_sentences:
        stri = ['\', "', '", \'', '\', \'', '", "']
        tmp = [row['Headlines'].find(stri[0], i), row['Headlines'].find(stri[1], i), row['Headlines'].find(stri[2], i), row['Headlines'].find(stri[3], i)]
        ans = 1e10
        for t in tmp:
            if t == -1:
                t = 1e10
            ans = min(ans, t)
        if ans == 1e10:
            ans = max_sentences - 2
        j = ans

        hea = row['Headlines'][i:j]
        i = j + 4
        if hea == '[' or hea == ']':
            continue
        row[f'{cnt}'] = hea
        cnt = cnt + 1
        print(hea)
    return row



---

### **Applying for NET Dataset**


---



In [ ]:
# Apply the function to split sentences
net = net.apply(split_sentences, axis=1)

In [ ]:
# Drop the original 'Headlines' column
net.drop(columns=['Headlines'], inplace=True)

In [ ]:
# Rename the 'old_name' column to 'new_name'
net = net.rename(columns={'Date': '0'})

In [ ]:
# Convert column names to integers and sort them numerically
net.columns = net.columns.astype(int)
net = net.reindex(sorted(net.columns), axis=1)

In [ ]:
net.to_csv('et_rev.csv')



---

### **Applying for NTOI Dataset**


---



In [ ]:
# Apply the function to split sentences
ntoi = ntoi.apply(split_sentences, axis=1)

In [ ]:
# Drop the original 'Headlines' column
ntoi.drop(columns=['Headlines'], inplace=True)

In [ ]:
# Rename the 'old_name' column to 'new_name'
ntoi = ntoi.rename(columns={'Date': '0'})

In [ ]:
# Convert column names to integers and sort them numerically
ntoi.columns = ntoi.columns.astype(int)
ntoi = ntoi.reindex(sorted(ntoi.columns), axis=1)

In [ ]:
ntoi.to_csv('toi_rev.csv')

---
## Discarding irrelevant News Headlines
---

```
A NLP model has been created and uploaded to Hugging Face Hub.
Now we have to use that model and discard some of the news headlines based on their output categories.

This way, we will only be using the news that's relevant and affects the price of our Stock.

In broad terms, these categories that we have to save are:

> FINANCE
> BUSINESS
> POLITICS
> WORLD NEWS
> SCIENCE & TECHNOLOGY
> LIFESTYLE
> HEALTH & MEDICINE

Others, we will discard.

```


In [ ]:
# Installing Transformers with sentencepiece
!pip install --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 309.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.3 MB/s eta 0:00:00



```
'HEALTH & MEDICINE': 0
'ENTERTAINMENT': 1
'FINANCE': 2
'SCIENCE & TECHNOLOGY': 3
'WORLD NEWS': 4
'EDUCATION': 5
'SOCIETY': 6
'POLITICS': 7
'ENVIRONMENT': 8
'BUSINESS': 9
'MEDIA': 10
'FOOD': 11
'CRIME': 12
'LIFESTYLE': 13
'SPORTS': 14
```

In [ ]:
from transformers import TFDistilBertForSequenceClassification
from sklearn.metrics import f1_score
from tqdm import tqdm
import pickle
from transformers import BertTokenizer, BertForSequenceClassification
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from tqdm import trange

In [ ]:
!huggingface-cli login

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

classifier = pipeline("text-classification", model="Yueh-Huan/news-category-classification-distilbert")

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at Yueh-Huan/news-category-classification-distilbert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [ ]:
output = classifier("Adani's stock rises by 10 dollars in a month.")

In [ ]:
print(output[0]['label'])

BUSINESS


In [ ]:
import pandas as pd

In [ ]:
net_rev = pd.read_csv('et_rev.csv')

In [ ]:
ntoi_rev = pd.read_csv('toi_rev.csv')

In [ ]:
list_save = ['MONEY', 'BUSINESS', 'POLITICS', 'WORLDPOST', 'U.S. NEWS', 'SCIENCE', 'TECH', 'MEDIA', 'THE WORLDPOST', 'WORLD NEWS', 'IMPACT']

###For ET Dataset
---

In [ ]:
save_tet = []
for i in range(len(net_rev)):
  dayz_save = []
  for col in range(1, len(net_rev.columns)-1):
    head = net_rev[f'{col}'][i]
    if(pd.isnull(head)):
      break
    op = classifier(head)[0]['label']
    print(op)
    if op in list_save:
      dayz_save.append([op])
  save_tet.append(dayz_save)


In [ ]:
saved_et = pd.DataFrame(save_tet)
saved_et

In [ ]:
saved_et.to_csv('saved_et.csv')

###For TOI Dataset
---

In [ ]:
save_ttoi = []
for i in range(len(ntoi_rev)):
  dayz_save = []
  for col in range(1, len(ntoi_rev.columns)-1):
    head = ntoi_rev[f'{col}'][i]
    if(pd.isnull(head)):
      break
    op = classifier(head)[0]['label']
    print(op)
    if op in list_save:
      dayz_save.append([op])
  save_ttoi.append(dayz_save)

In [ ]:
saved_toi = pd.DataFrame(save_ttoi)
saved_toi

In [ ]:
saved_toi.to_csv('saved_toi.csv')

```
Now we have two refined datasets, saved_et.csv and saved_toi.csv, which have news headlines that wew can ultimately use with any stock price.

Now, only the labels will change according to the name of stock.

Headlines will be the same!
```

---
##Merging the two datasets
---

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
saved_et = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/saved_et.csv')

<ipython-input-10-bfd98a0d13ac>:1: DtypeWarning: Columns (411,412,413,414,415,416,417,418,419,420) have mixed types. Specify dtype option on import or set low_memory=False.
  saved_et = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/saved_et.csv')


In [ ]:
saved_toi = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/saved_toi.csv')

<ipython-input-11-fe8deef41c96>:1: DtypeWarning: Columns (277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325) have mixed types. Specify dtype option on import or set low_memory=False.
  saved_toi = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/saved_toi.csv')


In [ ]:
# A = [['Date', 'col1', 'col2', 'col3', 'col4'],
#      [01-01-2013, 'val_a_1', 'val_a_2', np.nan, np.nan],
#      [02-01-2013, 'val_a_3', 'val_a_4', 'val_a_5', 'val_a_6'],
#      [03-01-2013, 'val_a_7', 'val_a_8', 'val_a_9', np.nan]]

# B = [['Date', 'col1', 'col2', 'col3', 'col4', 'col5'],
#      [01-01-2013, 'val_b_1', 'val_b_2', np.nan, np.nan, np.nan],
#      [02-01-2013, 'val_b_3', 'val_b_4', 'val_b_5', 'val_b_6', 'val_b_7'],
#      [03-01-2013, 'val_b_8', 'val_b_9', 'val_b_10', 'val_b_11', np.nan]]


# res = [['Date', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9'],
#      [01-01-2013, 'val_a_1', 'val_a_2', 'val_b_1', 'val_b_2', np.nan, np.nan, np.nan, np.nan, np.nan],
#      [02-01-2013, 'val_a_3', 'val_a_4', 'val_a_5', 'val_a_6', 'val_b_3', 'val_b_4', 'val_b_5', 'val_b_6', 'val_b_7'],
#      [03-01-2013, 'val_a_7', 'val_a_8', 'val_a_9', 'val_b_8', 'val_b_9', 'val_b_10', 'val_b_11', np.nan, np.nan]]

In [ ]:
import pandas as pd
import numpy as np

# Provided lists A and B
A = [['Date', 'col1', 'col2', 'col3', 'col4'],
     ['01-01-2013', 'val_a_1', 'val_a_2', np.nan, np.nan],
     ['02-01-2013', 'val_a_3', 'val_a_4', 'val_a_5', 'val_a_6'],
     ['03-01-2013', 'val_a_7', 'val_a_8', 'val_a_9', np.nan]]

B = [['Date', 'col1', 'col2', 'col3', 'col4', 'col5'],
     ['01-01-2013', 'val_b_1', 'val_b_2', np.nan, np.nan, np.nan],
     ['02-01-2013', 'val_b_3', 'val_b_4', 'val_b_5', 'val_b_6', 'val_b_7'],
     ['03-01-2013', 'val_b_8', 'val_b_9', 'val_b_10', 'val_b_11', np.nan]]

# Convert lists to DataFrames
df_A = pd.DataFrame(A[1:], columns=A[0])
df_B = pd.DataFrame(B[1:], columns=B[0])


In [ ]:
# Merge DataFrames on 'Date'
saved_merged = saved_et.merge(saved_toi, on='Unnamed: 0', how='inner')


In [ ]:
# Reorder columns so that NaNs are at the end
df_resultant = saved_merged.apply(lambda row: sorted(row, key=pd.isnull), axis=1)


###Revising this final Dataset

In [ ]:
res = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/res.csv')

In [ ]:
res['0'][0][len(res['0'][0])-2]

'n'

In [ ]:
def split_sentences(row):
    max_sentences = len(row['0'])
    cnt = 1
    i = 7
    while i < max_sentences:
        stri = ['\']', '"]']
        tmp = [row['0'].find(stri[0], i), row['0'].find(stri[1], i)]
        ans = 1e10
        for t in tmp:
            if t == -1:
                t = 1e10
            ans = min(ans, t)
        if ans == 1e10:
            break
        j = ans

        head = row['0'][i:j]
        i = j + 8
        if head == '[' or head == ']':
            continue
        row[f'{cnt}'] = head
        cnt = cnt + 1
        print(head)
    return row

In [ ]:
# Apply the function to split sentences
res = res.apply(split_sentences, axis=1)

Streaming output truncated to the last 5000 lines.
Tech Layoffs: Microsoft-owned GitHub to shed 10% employees, will close all offices
Oil prices jump more than 2% on Russian plan to cut output
Lupin Q3 Results: Profit slumps 72% to Rs 153 cr
Layoffs strip away tech worker visas along with jobs
Gold tumbles Rs 669; Silver plummets Rs 1,026
UP receives investment proposals of Rs 32.92 lakh crores through GIS roadshows: CM Yogi Adityanath
In India\'s well-being lies world prosperity: PM Modi at UP investors\' summit
2022 smartphone shipments fell 10% on year: IDC
4.34 crore hospital admissions costing Rs 51,749 crore authorised under govt\'s health insurance scheme: Health Minister
PPF, SCSS, Sukanya Samriddhi, other small savings schemes: What happens if an accountholder dies without nominee; govt clarifies
MSCI says four Adani stocks accounted for 0.27% of EM index as of Feb 8
Prime Minister Narendra Modi\'s popularity intact despite Adani controversy, says new poll
No impact on policyh

In [ ]:
# Drop the original 'Headlines' column
res.drop(columns=['0'], inplace=True)

In [ ]:
res.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
# Convert column names to integers and sort them numerically
res.columns = res.columns.astype(int)
res = res.reindex(sorted(res.columns), axis=1)

In [ ]:
res.to_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/res_rev.csv')

In [ ]:
res_rev = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/res_rev.csv')

<ipython-input-162-243e6dba9c0b>:1: DtypeWarning: Columns (637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709) have mixed types. Specify dtype option on import or set low_memory=False.
  res_rev = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/res_rev.csv')


In [ ]:
res_rev.replace({'â': "'", 'â': "'"}, regex=True, inplace=True)

In [ ]:
res_new = res_rev.replace(to_replace=r"\\", value="", regex=True)

In [ ]:
# Define a function to drop characters from the string based on the index
def drop_chars(cell, index):
    if index >= 10 and index < 100:
        return cell[1:]
    elif index >= 100 and index < 1000:
        return cell[2:]
    elif index >= 1000:
        return cell[3:]
    else:
        return cell


In [ ]:
# Apply the function to each cell of the 'col_1' column
res_new['1'] = [drop_chars(cell, idx) for idx, cell in enumerate(res_new['1'])]

In [ ]:
# Access a particular cell
cell_value = res_new.loc[8, '4']


In [ ]:
# Find out the data type of the cell
cell_dtype = type(cell_value)

In [ ]:
print(cell_dtype)

<class 'str'>


In [ ]:
res_new.columns

Index(['Unnamed: 0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '700', '701', '702', '703', '704', '705', '706', '707', '708', '709'],
      dtype='object', length=710)

In [ ]:
res_new.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
res_new.head(3)

,1,2,3,4,5,6,7,8,9,10,...,700,701,702,703,704,705,706,707,708,709
0,Direct cash transfer in 20 districts from today,Why Singapore scores over India on settlement ...,Sharp to insulate Indian operations from globa...,Anti-rape laws: BJP leader Sushma Swaraj reviv...,Government to allow power companies to divert ...,New year starts with a number of reasons to wo...,Govt proposes separate power supply lines for ...,B-schools strengthen family business courses t...,Higher investment limits & weaker rupee help F...,Gold set for 12th annual gain on stimulus efforts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Essar Engery in talks with lenders for crude i...,Indian student arrested in Singapore over bomb...,Natural gas futures fall on milder weather,Haryana Minister Shiv Charan Sharma stirs a ro...,ET review: Lenovo LePhone K860,Why 2013 will be a crucial year for chief econ...,New year: Seven jobs in finance up for grabs i...,Under the lens: Bankers to watch out for in 2013,Commodity broking firms doubled income from FI...,Sebi seeks approval for foreign funds into alt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Focus shifts to fiscal deficit as US House of ...,Sovereign debt of leading economies to fall in...,US 'fiscal cliff' deal puts higher tax on 77% ...,"Tata group to invest over Rs 45,000 cr, expand...","IT officials seize Rs 28,000 cr worth 'US bond...",Setback for Narendra Modi: SC upholds Justice ...,Delhi rape case should be tried speedily but l...,The Water Policy is at best a beginning and ne...,Ending oil subsidies key to curb a runaway cur...,How India Inc can make their CSR spends count,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
res_new.index.name = 'Index'

In [ ]:
res_new.head(3)

In [ ]:
res_new.to_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/res_new_without_date.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/res_new_without_date.csv')

<ipython-input-188-42ff05c9ff64>:1: DtypeWarning: Columns (637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/res_new_without_date.csv')


In [ ]:
import datetime
start_date = datetime.datetime(2013, 1, 1)

date_list = [start_date.date() + datetime.timedelta(days=x) for x in range(44976-41275)]
dates = [x.strftime("%d-%m-%Y") for x in date_list]

In [ ]:
len(dates)

3701

In [ ]:
df.head(2)

,Date,Unnamed: 0,1,2,3,4,5,6,7,8,...,700,701,702,703,704,705,706,707,708,709
0,01-01-2013,0,Direct cash transfer in 20 districts from today,Why Singapore scores over India on settlement ...,Sharp to insulate Indian operations from globa...,Anti-rape laws: BJP leader Sushma Swaraj reviv...,Government to allow power companies to divert ...,New year starts with a number of reasons to wo...,Govt proposes separate power supply lines for ...,B-schools strengthen family business courses t...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02-01-2013,1,Essar Engery in talks with lenders for crude i...,Indian student arrested in Singapore over bomb...,Natural gas futures fall on milder weather,Haryana Minister Shiv Charan Sharma stirs a ro...,ET review: Lenovo LePhone K860,Why 2013 will be a crucial year for chief econ...,New year: Seven jobs in finance up for grabs i...,Under the lens: Bankers to watch out for in 2013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Replace the 'Unnamed: 0' column with the new list
df.insert(loc=0, column='Date', value=dates)

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.head(1)

,Date,1,2,3,4,5,6,7,8,9,...,700,701,702,703,704,705,706,707,708,709
0,01-01-2013,Direct cash transfer in 20 districts from today,Why Singapore scores over India on settlement ...,Sharp to insulate Indian operations from globa...,Anti-rape laws: BJP leader Sushma Swaraj reviv...,Government to allow power companies to divert ...,New year starts with a number of reasons to wo...,Govt proposes separate power supply lines for ...,B-schools strengthen family business courses t...,Higher investment limits & weaker rupee help F...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(3701, 710)

In [ ]:
df.to_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/res_new_with_date.csv')



```
Dates added!
Datasets merged.
CSV Saved!
```



## Adding Sentiment Score

In [1]:
!pip install transformers

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", device=0)

In [3]:
import pandas as pd

In [6]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/news_dataset_for_financial_analysis.csv', index_col=False)

<ipython-input-6-962f1d451b17>:1: DtypeWarning: Columns (638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/news_dataset_for_financial_analysis.csv', index_col=False)


In [7]:
df = df.drop('Unnamed: 0', axis=1)

In [10]:
def scale_value(value):
    min_val = 0
    max_val = 1
    new_min = -0.1
    new_max = 0.1
    scaled_value = ((value - min_val) / (max_val - min_val)) * (new_max - new_min) + new_min
    return scaled_value

In [13]:
def give_score(row):
  score = 0.0
  cnt = 0
  for head in row:
    if pd.isna(head):
      break
    cnt = cnt + 1
    op = pipe(head)
    if op[0]['label'] == 'negative':
      score = score - op[0]['score']
    elif op[0]['label'] == 'neutral':
      score = score + scale_value(op[0]['score'])
    else:
      score = score + op[0]['score']
  avg = score/cnt
  print(avg, " ---", cnt)
  return avg

In [15]:
# Apply the function to each row
score_df = df[df.columns.difference(['Date'])].apply(give_score, axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


0.07631101063319619  --- 210
0.06903892306249536  --- 268
0.10368060415848758  --- 266
0.05455414466938727  --- 295
0.01839937402142419  --- 180
0.04579674105404473  --- 179
0.14371277847656838  --- 325
0.005016126138878511  --- 309
0.03585858637491863  --- 375
0.005733386907109465  --- 326
0.05712734037994318  --- 351
0.0484349380056542  --- 166
-0.0038242488914395005  --- 161
0.01618004828963238  --- 329
0.08339104515905597  --- 262
0.07288337009574718  --- 303
0.06550288665976996  --- 297
0.06753952552194473  --- 292
0.04426152774657326  --- 174
0.054570661693118395  --- 172
0.11196253755513358  --- 306
0.07812110253020649  --- 283
0.06961970307610252  --- 275
0.06450219376230092  --- 317
0.04969005515850871  --- 251
0.06146165442793334  --- 146
0.04013729632355787  --- 131
0.09319782142179558  --- 332
0.09157418525143037  --- 314
0.10800377433354032  --- 309
0.09388411883887558  --- 329
0.021271397865424726  --- 295
-0.014458273725900033  --- 171
-0.009606871179714326  --- 157
0.06

In [28]:
score_df = score_df.assign(Date = df['Date'])

In [32]:
score_df.to_csv('/content/drive/MyDrive/Dataset/Stock Price Prediction/News/sentiment_scores.csv')

---
---

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

---
---